In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import scipy.io as sio
from dataclasses import dataclass
from typing import List, Tuple
import os
from dotenv import load_dotenv
load_dotenv()
import tidy3d as td
from tidy3d import web
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import AutomationModule as AM

In [ ]:
tidy3dAPI = os.environ["API_ABRAHAM"]

In [ ]:
folder_path ="d:\phd stuff\structures\\18.01a"
project_name = "chi_test"
postprocess_results = []
runtime = 800
min_steps_per_lambda = 45
for direction in ["z"]: 
    for filename in os.listdir(folder_path):
        print(filename)
        if not (Path(filename).suffix==".h5"):
            continue 
        if os.path.isfile(os.path.join(folder_path, filename)):
            file=os.path.join(folder_path, filename)
            structure_1 = AM.loadAndRunStructure(key = tidy3dAPI, file_path=file
                                            ,direction=direction, lambda_range=[0.25,0.15],
                                            box_size=1.801,runtime=runtime,min_steps_per_lambda=min_steps_per_lambda,
                                           scaling=1,shuoff_condtion=1e-12, verbose=True, monitors=["flux"])
            
           
            print(structure_1.__str__())
            structure_1.plot_sim_layout()
            #print(structure_1.estimate_cost())
            
            results = structure_1.run_sim(folder_description=project_name,load=True,run_free=False)
            

            sim_data, sim_data0, task_name_def = results


            


In [ ]:
 # #Plot flux
#Retrieve the power flux through the monitor plane.
transmission0 = sim_data0['flux1'].flux
transmission = sim_data['flux1'].flux
transmission_normalized = transmission / transmission0
transmission_flux_right = sim_data['flux1'].flux
transmission_flux_left = sim_data['flux2'].flux
monitor_lambdas = td.C_0/np.array(np.array(sim_data.simulation.monitors)[0].freqs)
plt.semilogy(monitor_lambdas, (transmission_flux_left/transmission0), label='Left Monitor')
plt.semilogy(monitor_lambdas, (transmission_flux_right/transmission0), label='Right Monitor')
plt.xlabel('$\lambda (\mu m)$')
plt.ylabel('T')
plt.legend()
plt.grid()
plt.show()

In [ ]:
int_half = sim_data.get_intensity("time_monitorH")
int_end = sim_data.get_intensity("time_monitorFinal")
flux_f = sim_data["flux1"].flux
flux_t = sim_data["time_monitorT"].flux

In [ ]:
def take_log(intensity):
        return np.log10(intensity / np.max(intensity)+1e-2)

In [ ]:
#f, ((ax1, ax2)) = plt.subplots(1, 2, tight_layout=True)
#flux_f.plot(ax=ax1)
#flux_t.plot(ax=ax2)

take_log(int_half).interp(z=0).squeeze().plot.pcolormesh()
plt.show()
take_log(int_end).interp(z=0).squeeze().plot.pcolormesh()
plt.show()


In [ ]:
fig = plt.figure()
ax = plt.gca()
ax.scatter(flux_t['t'],flux_t)
plt.ylabel("Flux")
plt.xlabel("Time[ps]")
#plt.ylim(1e-17,1e-1)
ax.set_yscale("log")